In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from utils.evaluation_utils import load_data, save_data
import gc
import os

In [2]:
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if (col_type != object) or (col_type != Timestamp):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [5]:
# load data
data_filename = ["../data/raw_data/conformity_result1_2019_yura_1.csv", 
                 "../data/raw_data/conformity_result1_2020_yura_1.csv",
                 "../data/raw_data/conformity_result1_2020_yura_2.csv",
                 "../data/raw_data/conformity_result1_2023_yura_1.csv",
                 "../data/raw_data/conformity_result1_2023_yura_2.csv"]

data = []
for path in data_filename:
    per_data = pd.read_csv(path)
    
    if "2019" in path:
        year_code = [1,0,0]
    elif "2020" in path[0]:
        year_code = [0,1,0]
    elif "2023" in path[0]:
        year_code = [0,0,1]
    per_data[['year_0','year_1','year_2']] = np.tile(year_code, (len(per_data), 1))
    data.append(per_data)
    del per_data
    gc.collect()

data = pd.concat(data, axis=0, ignore_index=True)

# reduce mem
data = reduce_mem(data)
gc.collect()
data.rename(columns={"grid_id":"pid", "weight":"dist", "sc_250_avg":"sci"}, inplace=True)

8545.41 Mb, 2594.14 Mb (69.64 %)


In [6]:
# data sampling
sample_id = data['pid'].drop_duplicates().sample(n=10000)
data = data[data['pid'].isin(sample_id)]

In [7]:
# fill missing values
# 步骤 1: 分组并计算每个组中 'sci' 和 'dist' 的非缺失值数量
non_missing_counts = data.groupby('pid')[['sci','dist']].count()
# 步骤 2: 找出 'sci' 和 'dist' 都是完全缺失的组
groups_to_remove = non_missing_counts[(non_missing_counts['sci'] == 0) | (non_missing_counts['dist'] == 0)].index.get_level_values('pid').unique()
# 步骤 3: 生成要删除的行的索引
indexes_to_drop = data[data.set_index('pid').index.isin(groups_to_remove)].index
data = data.drop(indexes_to_drop)  # 使用 drop 删除这些行
# 步骤 4: 使用均值填充缺失值
data['sci'] = data.groupby('pid')['sci'].transform(lambda x: x.fillna(x.mean()))
data['dist'] = data.groupby('pid')['dist'].transform(lambda x: x.fillna(x.mean()))
# step 5: 重新排序
data = data.sort_values(by=['pid', 'date']).reset_index(drop=True)

In [9]:
# features join
# 1.read individual attributes
attr = pd.read_csv("../data/raw_data/sample_grid_attr_yura_1.csv")
attr.rename(columns = {"grid_count":"pid", "grid_g":"gender", "grid_type":"age"}, inplace=True)
be_varis = pd.read_csv("../data/raw_data/BE_varis.csv")
#attr = pd.merge(attr, be_varis[['fnid','density','landusemix','road_density','center','subway','edu','married','rent']], on='fnid',how="left")
attr = pd.merge(attr, be_varis[['fnid','edu','married','rent']], on='fnid',how="left")
# 对没有fnid记录的样本，取全样本均值填充缺失值
attr.fillna(attr.mean(), inplace=True)

data = pd.merge(data, attr.drop(columns=['date']), on='pid',how="left")

In [10]:
# 2. individual travel attributes
spatial_ent = pd.read_csv("../data/raw_data/sample_poi_month_entro_yura.csv")
spatial_ent.rename(columns={"grid_id":"pid", "entropy":"spatial_ent"}, inplace=True)
temperal_ent = pd.read_csv("../data/raw_data/sample_move_month_entro_yura.csv")
temperal_ent.rename(columns={"grid_id":"pid", "entropy":"temperal_ent"}, inplace=True)
# merge them
data['year_month'] = (data['date']/100).astype('int')
data = pd.merge(data, spatial_ent, on=['pid','year_month'], how="left")
data = pd.merge(data, temperal_ent, on=['pid','year_month'], how="left")
# fill 0
data['spatial_ent'] = data['spatial_ent'].fillna(0)
data['temperal_ent'] = data['temperal_ent'].fillna(0)

In [11]:
# 3.case data
case = pd.read_csv("../data/raw_data/case_series.csv")
data = pd.merge(data, case[['date','voluntary']], on="date", how="left")
data["voluntary"] = data["voluntary"].fillna(0)

In [12]:
# 4.weather data
weather = pd.read_csv("../data/raw_data/weather_info.csv")
data = pd.merge(data, weather, on="date", how="left")

In [13]:
# features construction
data['weekday'] = pd.to_datetime(data['date'].astype(str), format="%Y%m%d").dt.weekday
holidays_conditions = (
(data['date'].isin([20190101, 20200101])) |
(data['date'].between(20190204, 20190210) | data['date'].between(20200124, 20200202)) |
(data['date'].between(20190405, 20190407) | data['date'].between(20200404, 20200406) | (data['date'] == 20230405)) |
(data['date'].between(20190501, 20190504) | data['date'].between(20200501, 20200505) | data['date'].between(20230429, 20230503)) |
(data['date'].between(20230622, 20230624)))

data['festivals'] = np.where(holidays_conditions, 1, 0)

In [14]:
# policy variables (E1 & E2)
date_conditions = [
    (data['date'].between(20200124, 20200223)),
    (data['date'].between(20200224, 20200508)),
    #(data['date'].between(20200509, 20201231))
]
event_columns = ['E1','E2']
for col, cond in zip(event_columns, date_conditions):
    data[col] = np.where(cond, 1, 0)

In [15]:
# individual attributes binning
income_thre = data['rent'].quantile(q=[0, 0.2, 0.4, 0.6, 0.8, 1])
data['age'] = pd.cut(data['age'], bins=[0, 5, 13, 16, float('inf')], labels=['Young', 'Adult', 'MiddleElder', 'Elder'], right=False)
data['gender'] = data['gender'].map({1: 'Male', 2: 'Female'})
data['income'] = pd.cut(data['rent'], bins=[0] + list(income_thre)[1:], labels=['Bottom', 'Lower', 'Middle', 'Upper', 'Superior'], right=True, include_lowest=True)

# Converting columns to category type with specific levels
data['age'] = data['age'].astype('category')
data['gender'] = data['gender'].astype('category')
data['income'] = pd.Categorical(data['income'], categories=["Bottom", "Lower", "Middle", "Upper", "Superior"], ordered=True)

le = LabelEncoder()
ind_attr_cols = ['gender', 'age', 'income']
for col in ind_attr_cols:
    data[col] = le.fit_transform(data[col])

In [16]:
# weekday one-hot encoding
weekday_encoded = pd.get_dummies(data['weekday'], prefix='weekday').astype(int)
data = data.drop('weekday', axis=1)
data= pd.concat([data, weekday_encoded], axis=1)

In [17]:
 # treatment and outcome - log transformation
data['sci'] = np.log(data['sci'] + 1)
data['dist'] = np.log(data['dist'] + 1)
data['voluntary'] = np.log(data['voluntary'] + 1)

# shift获得t-1列（voluntary是否需要t-1）
# use yesterday norm_mean as conformity factor
data[['sci_yes','dist_yes','voluntary_yes']] = data.groupby('pid')[['sci','dist','voluntary']].shift(1)
# delete null in conformity and dist_yes (i.e. delete the first day in every year)
data = data.dropna(subset=['sci_yes','dist_yes','voluntary_yes']).reset_index(drop=True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4880000 entries, 0 to 4879999
Data columns (total 33 columns):
 #   Column         Dtype  
---  ------         -----  
 0   pid            int32  
 1   date           int32  
 2   dist           float16
 3   sci            float32
 4   year_0         int8   
 5   year_1         int8   
 6   year_2         int8   
 7   gender         int64  
 8   age            int64  
 9   fnid           float64
 10  edu            float64
 11  married        float64
 12  rent           float64
 13  year_month     int64  
 14  spatial_ent    float64
 15  temperal_ent   float64
 16  voluntary      float64
 17  temperature    float64
 18  percipit       float64
 19  festivals      int64  
 20  E1             int64  
 21  E2             int64  
 22  income         int64  
 23  weekday_0      int64  
 24  weekday_1      int64  
 25  weekday_2      int64  
 26  weekday_3      int64  
 27  weekday_4      int64  
 28  weekday_5      int64  
 29  weekday_6     

In [19]:
data.head()

,pid,date,dist,sci,year_0,year_1,year_2,gender,age,fnid,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,sci_yes,dist_yes,voluntary_yes
0,12023,20190102,2.017578,4.242451,1,0,0,0,0,25498.0,...,0,0,1,0,0,0,0,7.425946,2.244141,0.0
1,12023,20190103,0.000000,7.956927,1,0,0,0,0,25498.0,...,0,0,0,1,0,0,0,4.242451,2.017578,0.0
2,12023,20190104,0.000000,7.206891,1,0,0,0,0,25498.0,...,0,0,0,0,1,0,0,7.956927,0.000000,0.0
3,12023,20190105,2.671875,5.529972,1,0,0,0,0,25498.0,...,0,0,0,0,0,1,0,7.206891,0.000000,0.0
4,12023,20190106,0.000000,5.490638,1,0,0,0,0,25498.0,...,0,0,0,0,0,0,1,5.529972,2.671875,0.0


In [20]:
data.to_csv("../data/sample_1w_dataframe.csv", index=False)

In [2]:
data = pd.read_csv("../data/raw_data/sample_mobile_flow_rango.csv")
data.head()

,grid_id,yearmonth,month_duration,month_ltime
0,69566445,20230801,98993,8240.0
1,69564867,20230801,57113,129749981.0
2,69563436,20230501,31624,267612265.0
3,69561851,20190401,37140,NaN
4,69561658,20230801,8408,345302597.0


In [3]:
data['month_ltime'] = data['month_ltime'].fillna(0)
data['flow'] = data['month_ltime'] / data['month_duration']

## 数据集处理
1. 删掉多余的建成环境指标
2. 生成三年的数据，其中treatment只有sci，policy作为协变量
3. 提取出20年单独的数据

In [2]:
dataset = load_data("../data/sample_32w.h5")

for key in dataset.keys():
    print(key)
    print(dataset[key].shape)
    print(dataset[key].dtype)

covariates
(981651, 161, 23)
float32
outcomes
(981651, 161, 1)
float32
sequence_length
(981651,)
int64
treatments
(981651, 161, 1)
float32


原始的covariate列
['gender','age','income','weekday_0','weekday_1','weekday_2','weekday_3','weekday_4', 'weekday_5', 'weekday_6',
                'voluntary','festivals','year_0','year_1','year_2','density','landusemix','road_density','center','subway','edu',
                'married','dist_yes','temperature','percipit']

In [4]:
# 列表中 'landusemix', 'road_density', 'center', 'subway' 的索引
columns_to_remove = [16, 17, 18, 19]  # 以0为起始索引

# 删除 covariates 中的指定列
dataset['covariates'] = np.delete(dataset['covariates'], columns_to_remove, axis=2)

# 删除 previous_covariates 中的指定列
dataset['previous_covariates'] = np.delete(dataset['previous_covariates'], columns_to_remove, axis=2)

for key in dataset.keys():
    print(key)
    print(dataset[key].shape)
    print(dataset[key].dtype)

covariates
(981651, 161, 21)
float32
outcomes
(981651, 161, 1)
float32
previous_covariates
(981651, 160, 21)
float32
previous_treatments
(981651, 160, 3)
float32
sequence_length
(981651,)
int64
treatments
(981651, 161, 3)
float32


此时的covariate列
['gender','age','income','weekday_0','weekday_1','weekday_2','weekday_3','weekday_4', 'weekday_5', 'weekday_6',
                'voluntary','festivals','year_0','year_1','year_2','density','edu',
                'married','dist_yes','temperature','percipit']

In [5]:
# 提取出2020年单独的数据，生成2020data
def obtain_dataset(dataset, column_index, selection=1):
    dataset_part = {}
    selected_indices = np.where(dataset['covariates'][:, :, column_index] == selection)[0]
    unique_selected_indices = np.unique(selected_indices)
    for key in dataset.keys():
        dataset_part[key] = dataset[key][unique_selected_indices]
    return dataset_part

column_2020 = 13  # index of year 2020 indicator
dataset20 = obtain_dataset(dataset, column_2020)

In [6]:
# 将政策变量加入covariate和previous_covariates
policy_columns = dataset['treatments'][:, :, -2:]

# 将这两列合并到 covariates 的最后
dataset['covariates'] = np.concatenate((dataset['covariates'], policy_columns), axis=2)

# 从 treatments 删除最后两列
dataset['treatments'] = dataset['treatments'][:, :, :-2]

for key in dataset.keys():
    print(key)
    print(dataset[key].shape)
    print(dataset[key].dtype)

covariates
(981651, 161, 23)
float32
outcomes
(981651, 161, 1)
float32
previous_covariates
(981651, 160, 21)
float32
previous_treatments
(981651, 160, 3)
float32
sequence_length
(981651,)
int64
treatments
(981651, 161, 1)
float32


此时的covariate列 ['gender','age','income','weekday_0','weekday_1','weekday_2','weekday_3','weekday_4', 'weekday_5', 'weekday_6', 'voluntary','festivals','year_0','year_1','year_2','density','edu', 'married','dist_yes','temperature','percipit', 'policy_1', 'policy_2']

In [8]:
# 删除 previous_treatments 键和其数据
if 'previous_treatments' in dataset:
    del dataset['previous_treatments']

# 删除 previous_covariates 键和其数据
if 'previous_covariates' in dataset:
    del dataset['previous_covariates']

for key in dataset.keys():
    print(key)
    print(dataset[key].shape)
    print(dataset[key].dtype)

covariates
(981651, 161, 23)
float32
outcomes
(981651, 161, 1)
float32
sequence_length
(981651,)
int64
treatments
(981651, 161, 1)
float32


In [9]:
# 保存生成的dataset20数据
save_data('../data/sample_32w_2020.h5', dataset20)

# 保存新的dataset
save_data('../data/sample_32w.h5', dataset)

Finish ../data/sample_32w_2020.h5!
Finish ../data/sample_32w.h5!


In [3]:
ps = np.load("results/rmsn_all_data_no_deconfound_use_confounders_False/propensity_scores.npy")
print(ps)

[[[ 1.        ]
  [ 0.2186165 ]
  [ 0.24198702]
  ...
  [ 0.60612154]
  [ 0.61547446]
  [ 0.64035958]]

 [[ 1.        ]
  [ 0.39805797]
  [ 0.71758068]
  ...
  [ 0.38854495]
  [ 0.61221683]
  [ 4.38675785]]

 [[ 1.        ]
  [ 2.76691246]
  [ 0.78857583]
  ...
  [ 0.36140603]
  [ 0.2374164 ]
  [ 0.29409248]]

 ...

 [[ 1.        ]
  [ 0.48634303]
  [ 4.63589239]
  ...
  [42.00141525]
  [ 2.19222713]
  [20.19682884]]

 [[ 1.        ]
  [ 1.60044539]
  [ 2.99170256]
  ...
  [ 2.70778584]
  [ 0.38779899]
  [ 9.78914738]]

 [[ 1.        ]
  [ 0.17965211]
  [ 0.1899278 ]
  ...
  [ 0.23001415]
  [ 0.25791985]
  [ 0.26360363]]]


In [3]:
dataset20 = load_data("../data/sample_32w_2020.h5")
for key in dataset20.keys():
    print(key)
    print(dataset20[key].shape)
    print(dataset20[key].dtype)

covariates
(327217, 161, 21)
float32
outcomes
(327217, 161, 1)
float32
previous_covariates
(327217, 160, 21)
float32
previous_treatments
(327217, 160, 3)
float32
sequence_length
(327217,)
int64
treatments
(327217, 161, 3)
float32
